## Data check

In [1]:
import librosa.display
from td_utils import *
%matplotlib inline

audio_dir = os.path.join('.','data','train_audios')
wav_file = 'mix_0.wav'
fullname = os.path.join(audio_dir, wav_file)

x = graph_spectrogram(fullname)
_, data = get_wav_info(fullname)
print("Time steps in audio recording before spectrogram", data.shape)
print("Time steps in input after spectrogram", x.shape)

librosa.display.specshow(x, sr=44100, x_axis='time', y_axis='mel')

c:\Users\user\anaconda3\envs\Beeeeep_2\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Time steps in audio recording before spectrogram (441000,)


AttributeError: 'tuple' object has no attribute 'shape'

In [14]:
n_freq = 128 # 스펙토그램 높이

# DataLoader

with keras

In [3]:
import os 
import pandas as pd
import numpy as np
from keras.utils import Sequence

In [4]:
data_dir = os.path.join('.','data', 'XY_train')
x_s = []
y_s = []
for file in os.listdir(data_dir):
    if file.startswith('x_'):
        x_s.append(os.path.join(data_dir,file))
    elif file.startswith('y_'):
        y_s.append(os.path.join(data_dir,file))
    x_s = sorted(x_s)
    y_s = sorted(y_s)
df = pd.DataFrame({'x':x_s, 'y':y_s})
df.head()

,x,y
0,.\data\XY_train\x_0.npy,.\data\XY_train\y_0.npy
1,.\data\XY_train\x_1.npy,.\data\XY_train\y_1.npy
2,.\data\XY_train\x_2.npy,.\data\XY_train\y_2.npy
3,.\data\XY_train\x_3.npy,.\data\XY_train\y_3.npy
4,.\data\XY_train\x_4.npy,.\data\XY_train\y_4.npy


In [5]:
train_ratio = 0.8
idxs = list(range(len(df)))
np.random.shuffle(idxs)
train_idx = idxs[:int(len(df)*train_ratio)]
valid_idx = idxs[int(len(df)*train_ratio):]

train_df = df.loc[train_idx]
valid_df = df.loc[valid_idx]

print(train_df.shape, valid_df.shape)

(8, 2) (2, 2)


In [6]:
data_dir = os.path.join('.','data', 'XY_test')
x_s = []
y_s = []
for file in os.listdir(data_dir):
    if file.startswith('x_'):
        x_s.append(os.path.join(data_dir,file))
    elif file.startswith('y_'):
        y_s.append(os.path.join(data_dir,file))
    x_s = sorted(x_s)
    y_s = sorted(y_s)
test_df = pd.DataFrame({'x':x_s, 'y':y_s})
test_df.head()

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '.\\data\\XY_test'

In [7]:
class DataGenerator(Sequence):
    def __init__(self, df, batch_size, shuffle = True):
        self.X = list(df.x)
        self.y = list(df.y)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.X))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __len__(self):
        return int(np.floor(len(self.X) / self.batch_size))
    
    def __data_generation(self, X_list, y_list):
        X = []
        y = []
        for i, (img, label) in enumerate(zip(X_list, y_list)):
            X.append(np.load(img))
            y.append(np.load(label))
        
        X = np.stack(X, axis=0)
        y = np.stack(y, axis=0)

        return X, y
        
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        X_list = [self.X[k] for k in indexes]
        y_list = [self.y[k] for k in indexes]
        X, y = self.__data_generation(X_list, y_list)
        return X, y

In [9]:
train_generator = DataGenerator(train_df, 5)
valid_generator = DataGenerator(valid_df, 5)
# test_generator = DataGenerator(test_df, 3)

# Train

In [10]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

In [11]:
def make_model(input_shape):
    
    X_input = Input(shape = input_shape)
    X = Conv1D(196, kernel_size=15, strides=4)(X_input)         # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                   # ReLu activation
    X = Dropout(0.8)(X)                                         # dropout (use 0.8)

    X = GRU(units = 128, return_sequences = True)(X)            # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                         # dropout (use 0.8)
    X = BatchNormalization()(X)                                 # Batch normalization

    X = GRU(units = 128, return_sequences = True)(X)            # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                         # dropout (use 0.8)
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Dropout(0.8)(X)                                         # dropout (use 0.8)

    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X)    # time distributed  (sigmoid)

    model = Model(inputs = X_input, outputs = X)
    return model  

In [15]:
model = make_model(input_shape = (None, n_freq))

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 128)]       0         
                                                                 
 conv1d (Conv1D)             (None, None, 196)         376516    
                                                                 
 batch_normalization (BatchN  (None, None, 196)        784       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, None, 196)         0         
                                                                 
 dropout (Dropout)           (None, None, 196)         0         
                                                                 
 gru (GRU)                   (None, None, 128)         125184    
                                                             

In [17]:
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

c:\Users\user\anaconda3\envs\Beeeeep_2\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [18]:
model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs = 10)

Epoch 1/10


c:\Users\user\anaconda3\envs\Beeeeep_2\lib\site-packages\ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


1/1 [==============================] - 4s 4s/step - loss: 1.4109 - accuracy: 0.5123
Epoch 2/10
1/1 [==============================] - 0s 204ms/step - loss: 1.4620 - accuracy: 0.4981
Epoch 3/10
1/1 [==============================] - 0s 231ms/step - loss: 1.4786 - accuracy: 0.5057
Epoch 4/10
1/1 [==============================] - 0s 197ms/step - loss: 1.4300 - accuracy: 0.4915
Epoch 5/10
1/1 [==============================] - 0s 199ms/step - loss: 1.4311 - accuracy: 0.5038
Epoch 6/10
1/1 [==============================] - 0s 191ms/step - loss: 1.4205 - accuracy: 0.5075
Epoch 7/10
1/1 [==============================] - 0s 193ms/step - loss: 1.4490 - accuracy: 0.4868
Epoch 8/10
1/1 [==============================] - 0s 195ms/step - loss: 1.3489 - accuracy: 0.5208
Epoch 9/10
1/1 [==============================] - 0s 195ms/step - loss: 1.5009 - accuracy: 0.4594
Epoch 10/10
1/1 [==============================] - 0s 195ms/step - loss: 1.4407 - accuracy: 0.4774


# Test

In [4]:
def postprocessing(outputs, th):
    for output in outputs:
        output[output<th] = 0
        output[output>=th] = 1
    return outputs

In [ ]:
import matplotlib.pyplot as plt
for batch in test_generator:
    x, y = batch
    output = model.predict(x)
    output = postprocessing(output,0.6)
    for i in range(len(y)):
        plt.plot(y[i], label='true')
        plt.plot(output[i], label='predict')
        plt.legend()
        plt.show()